In [ ]:
# File: trapped.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import os

import numpy as np
import sdf
from frontend import App

app = App.create("trapped")
filepath = "/tmp/squishy.tmp.ply"

# create squishy ball mesh if not exists
if not os.path.exists(filepath):
    V, F = app.mesh.icosphere(r=2, subdiv_count=2)
    func = sdf.sphere(1.1)
    for f in F:
        d = np.mean(V[f], axis=0)
        if d[0] > 0:
            func = func | sdf.capsule(-d, d, 0.05)
    func.save(filepath, step=0.03)

V, F, T = (
    app.mesh.load_tri(filepath)
    .decimate(100000)
    .tetrahedralize()
    .normalize()
    .scale(0.97)
)
app.asset.add.tet("squishy", V, F, T)

scene = app.scene.create()
(
    scene.add.invisible.sphere([0, 0, 0], 0.98)
    .invert()
    .radius(0.4, 2)
    .radius(0.4, 3)
    .radius(10, 4)
)
scene.add("squishy").at(0.5, 0, 0).jitter()
scene.add("squishy").at(-0.5, 0, 0).jitter()

scene = scene.build().report()
scene.preview()

In [ ]:
session = app.session.create(scene)
(
    session.param.set("gravity", 0.0)
    .set("csrmat-max-nnz", 3000000)
    .set("dt", 0.01)
)
session.param.dyn("playback").time(2.99).hold().time(3).change(0.1)
session = session.build()

In [ ]:
session.start().preview()
session.stream()

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()